In [81]:
import pandas as pd
import datetime

In [108]:
df = pd.read_csv('../datasets/nobel.csv')
df.head()

,awardYear,category,categoryFullName,sortOrder,portion,prizeAmount,prizeAmountAdjusted,dateAwarded,prizeStatus,motivation,...,org_founded_country,org_founded_countryNow,org_founded_locationString,ind_or_org,residence_1,residence_2,affiliation_1,affiliation_2,affiliation_3,affiliation_4
0,2001,Economic Sciences,The Sveriges Riksbank Prize in Economic Scienc...,2,1/3,10000000,12295082,2001-10-10,received,for their analyses of markets with asymmetric ...,...,NaN,NaN,NaN,Individual,NaN,NaN,"Stanford University, Stanford, CA, USA",NaN,NaN,NaN
1,1975,Physics,The Nobel Prize in Physics,1,1/3,630000,3404179,1975-10-17,received,for the discovery of the connection between co...,...,NaN,NaN,NaN,Individual,NaN,NaN,"Niels Bohr Institute, Copenhagen, Denmark",NaN,NaN,NaN
2,2004,Chemistry,The Nobel Prize in Chemistry,1,1/3,10000000,11762861,2004-10-06,received,for the discovery of ubiquitin-mediated protei...,...,NaN,NaN,NaN,Individual,NaN,NaN,"Technion - Israel Institute of Technology, Hai...",NaN,NaN,NaN
3,1982,Chemistry,The Nobel Prize in Chemistry,1,1,1150000,3102518,1982-10-18,received,for his development of crystallographic electr...,...,NaN,NaN,NaN,Individual,NaN,NaN,"MRC Laboratory of Molecular Biology, Cambridge...",NaN,NaN,NaN
4,1979,Physics,The Nobel Prize in Physics,2,1/3,800000,2988048,1979-10-15,received,for their contributions to the theory of the u...,...,NaN,NaN,NaN,Individual,NaN,NaN,"International Centre for Theoretical Physics, ...","Imperial College, London, United Kingdom",NaN,NaN


(1) In which years was there a price related to refugees?

In [86]:
df[df.motivation.str.contains('refugee')][['awardYear','category']]

,awardYear,category
267,1922,Peace
303,1958,Peace
615,1938,Peace
634,1954,Peace
948,1981,Peace


(2) In all categories there have been more men than women winners. In which category is the gap the largest (in absolute values)? And in percentage?

In [87]:
df.gender.isna().sum()

27

In [88]:
df.groupby(['category','gender'],dropna=False).size()

category                gender
Chemistry               female      5
                        male      179
Economic Sciences       female      2
                        male       82
Literature              female     15
                        male      101
Peace                   female     17
                        male       90
                        NaN        27
Physics                 female      3
                        male      210
Physiology or Medicine  female     12
                        male      207
dtype: int64

In [89]:
grouped = df.groupby(['category','gender'],dropna=False).size().\
    reset_index(name = 'count')
grouped.head()

,category,gender,count
0,Chemistry,female,5
1,Chemistry,male,179
2,Economic Sciences,female,2
3,Economic Sciences,male,82
4,Literature,female,15


---- Método (I) ----

In [6]:
def dif_f_m(df_in, ret = "dif_abs"):
    '''
    For a given category, extract difference of male - female awardees.
    
    Parameters
    ------------
    df_in: (DataFrame) DataFrame containing a 'gender' column (male, female, NA)
    and a 'count' column, with the number of awardees per gender.
    
    ret: (str) controla the output
    
    Returns
    ------------
    If ret == "dif_abs": (int) Absolute difference in magnitude
    
    Si ret == "dif_porc": (float) Difference in percentage
    '''
    f = df_in[df_in['gender'] == "female"]['count'].item()
    m = df_in[df_in['gender'] == "male"]['count'].item()
    tot = df_in['count'].sum()
    if(ret == "dif_abs"):
        return m-f
    elif(ret =="dif_porc"):
        return 100*(m-f)/tot
    else:
        return None

In [90]:
grouped

,category,gender,count
0,Chemistry,female,5
1,Chemistry,male,179
2,Economic Sciences,female,2
3,Economic Sciences,male,82
4,Literature,female,15
5,Literature,male,101
6,Peace,female,17
7,Peace,male,90
8,Peace,NaN,27
9,Physics,female,3


In [93]:
dif_m_f = grouped.groupby('category').\
    apply(lambda x: dif_f_m(x)).\
    reset_index(name ="abs_dif_m_f")
dif_m_f

,category,abs_dif_m_f
0,Chemistry,174
1,Economic Sciences,80
2,Literature,86
3,Peace,73
4,Physics,207
5,Physiology or Medicine,195


In [94]:
grouped.groupby('category').apply(lambda x: dif_f_m(x, ret = "dif_porc"))

category
Chemistry                 94.565217
Economic Sciences         95.238095
Literature                74.137931
Peace                     54.477612
Physics                   97.183099
Physiology or Medicine    89.041096
dtype: float64

In [95]:
dif_m_f['porc_dif_m_f'] = grouped.groupby('category').apply(lambda x: dif_f_m(x, ret = "dif_porc")).values
dif_m_f

,category,abs_dif_m_f,porc_dif_m_f
0,Chemistry,174,94.565217
1,Economic Sciences,80,95.238095
2,Literature,86,74.137931
3,Peace,73,54.477612
4,Physics,207,97.183099
5,Physiology or Medicine,195,89.041096


---- Método (II) ----

In [12]:
piv = grouped.pivot(index="category", columns="gender", values="count")
display(piv)

gender,NaN,female,male
category,,,
Chemistry,NaN,5.0,179.0
Economic Sciences,NaN,2.0,82.0
Literature,NaN,15.0,101.0
Peace,27.0,17.0,90.0
Physics,NaN,3.0,210.0
Physiology or Medicine,NaN,12.0,207.0


In [13]:
piv.rename(columns={ piv.columns[0]: "not_assigned" }, inplace = True)
display(piv)

gender,not_assigned,female,male
category,,,
Chemistry,NaN,5.0,179.0
Economic Sciences,NaN,2.0,82.0
Literature,NaN,15.0,101.0
Peace,27.0,17.0,90.0
Physics,NaN,3.0,210.0
Physiology or Medicine,NaN,12.0,207.0


In [14]:
piv['total'] = piv.sum(axis=1, skipna = True)
piv['abs_dif_m_f'] = piv.male - piv.female
display(piv)


gender,not_assigned,female,male,total,abs_dif_m_f
category,,,,,
Chemistry,NaN,5.0,179.0,184.0,174.0
Economic Sciences,NaN,2.0,82.0,84.0,80.0
Literature,NaN,15.0,101.0,116.0,86.0
Peace,27.0,17.0,90.0,134.0,73.0
Physics,NaN,3.0,210.0,213.0,207.0
Physiology or Medicine,NaN,12.0,207.0,219.0,195.0


In [15]:
piv['porc_dif_m_f'] = 100*piv.abs_dif_m_f/piv.total
display(piv)

gender,not_assigned,female,male,total,abs_dif_m_f,porc_dif_m_f
category,,,,,,
Chemistry,NaN,5.0,179.0,184.0,174.0,94.565217
Economic Sciences,NaN,2.0,82.0,84.0,80.0,95.238095
Literature,NaN,15.0,101.0,116.0,86.0,74.137931
Peace,27.0,17.0,90.0,134.0,73.0,54.477612
Physics,NaN,3.0,210.0,213.0,207.0,97.183099
Physiology or Medicine,NaN,12.0,207.0,219.0,195.0,89.041096


--- Comparación métodos ---

In [16]:
piv.sort_values('abs_dif_m_f', ascending = False)

gender,not_assigned,female,male,total,abs_dif_m_f,porc_dif_m_f
category,,,,,,
Physics,NaN,3.0,210.0,213.0,207.0,97.183099
Physiology or Medicine,NaN,12.0,207.0,219.0,195.0,89.041096
Chemistry,NaN,5.0,179.0,184.0,174.0,94.565217
Literature,NaN,15.0,101.0,116.0,86.0,74.137931
Economic Sciences,NaN,2.0,82.0,84.0,80.0,95.238095
Peace,27.0,17.0,90.0,134.0,73.0,54.477612


In [96]:
dif_m_f.sort_values('abs_dif_m_f', ascending = False)

,category,abs_dif_m_f,porc_dif_m_f
4,Physics,207,97.183099
5,Physiology or Medicine,195,89.041096
0,Chemistry,174,94.565217
2,Literature,86,74.137931
1,Economic Sciences,80,95.238095
3,Peace,73,54.477612


In [18]:
piv.sort_values('porc_dif_m_f', ascending = False)

gender,not_assigned,female,male,total,abs_dif_m_f,porc_dif_m_f
category,,,,,,
Physics,NaN,3.0,210.0,213.0,207.0,97.183099
Economic Sciences,NaN,2.0,82.0,84.0,80.0,95.238095
Chemistry,NaN,5.0,179.0,184.0,174.0,94.565217
Physiology or Medicine,NaN,12.0,207.0,219.0,195.0,89.041096
Literature,NaN,15.0,101.0,116.0,86.0,74.137931
Peace,27.0,17.0,90.0,134.0,73.0,54.477612


In [19]:
dif_m_f.sort_values('porc_dif_m_f', ascending = False)

,category,abs_dif_m_f,porc_dif_m_f
4,Physics,207,97.183099
1,Economic Sciences,80,95.238095
0,Chemistry,174,94.565217
5,Physiology or Medicine,195,89.041096
2,Literature,86,74.137931
3,Peace,73,54.477612


(3)

(a)

In [97]:
df.awardYear.min()

1901

In [98]:
begin_category = df.groupby('category')['awardYear'].min().reset_index()
begin_category

,category,awardYear
0,Chemistry,1901
1,Economic Sciences,1969
2,Literature,1901
3,Peace,1901
4,Physics,1901
5,Physiology or Medicine,1901


In [99]:
begin_category[begin_category.awardYear > df.awardYear.min()].category

1    Economic Sciences
Name: category, dtype: object

(4)

In [23]:
df.dtypes

awardYear                      int64
category                      object
categoryFullName              object
sortOrder                      int64
portion                       object
prizeAmount                    int64
prizeAmountAdjusted            int64
dateAwarded                   object
prizeStatus                   object
motivation                    object
categoryTopMotivation         object
award_link                    object
id                             int64
name                          object
knownName                     object
givenName                     object
familyName                    object
fullName                      object
penName                       object
gender                        object
laureate_link                 object
birth_date                    object
birth_city                    object
birth_cityNow                 object
birth_continent               object
birth_country                 object
birth_countryNow              object
b

In [100]:
(df.birth_countryNow == 'Austria').sum() #

18

In [101]:
df[df.birth_countryNow == 'Austria'].birth_country

36             Austria
201            Austria
218            Austria
230            Austria
269            Austria
497            Austria
502    Austrian Empire
503            Austria
520            Austria
572            Austria
585            Austria
683            Austria
719    Austria-Hungary
726    Austrian Empire
733            Austria
884            Austria
892            Austria
933            Austria
Name: birth_country, dtype: object

In [109]:
df[df.birth_date.str[5:7] == '00']

,awardYear,category,categoryFullName,sortOrder,portion,prizeAmount,prizeAmountAdjusted,dateAwarded,prizeStatus,motivation,...,org_founded_country,org_founded_countryNow,org_founded_locationString,ind_or_org,residence_1,residence_2,affiliation_1,affiliation_2,affiliation_3,affiliation_4
0,2001,Economic Sciences,The Sveriges Riksbank Prize in Economic Scienc...,2,1/3,10000000,12295082,2001-10-10,received,for their analyses of markets with asymmetric ...,...,NaN,NaN,NaN,Individual,NaN,NaN,"Stanford University, Stanford, CA, USA",NaN,NaN,NaN
26,1960,Peace,The Nobel Peace Prize,1,1,225987,2664941,1961-10-23,received,for his non-violent struggle against apartheid,...,NaN,NaN,NaN,Individual,South Africa,NaN,NaN,NaN,NaN,NaN
612,2018,Peace,The Nobel Peace Prize,2,1/2,9000000,9000000,2018-10-05,received,for their efforts to end the use of sexual vio...,...,NaN,NaN,NaN,Individual,NaN,NaN,NaN,NaN,NaN,NaN
670,2018,Economic Sciences,The Sveriges Riksbank Prize in Economic Scienc...,2,1/2,9000000,9000000,NaN,received,for integrating technological innovations into...,...,NaN,NaN,NaN,Individual,NaN,NaN,"NYU Stern School of Business, New York, NY, USA",NaN,NaN,NaN
785,2011,Physics,The Nobel Prize in Physics,1,1/2,10000000,10545557,2011-10-04,received,for the discovery of the accelerating expansio...,...,NaN,NaN,NaN,Individual,NaN,NaN,"Lawrence Berkeley National Laboratory, Berkele...","University of California, Berkeley, CA, USA",NaN,NaN
880,2009,Chemistry,The Nobel Prize in Chemistry,1,1/3,10000000,10958504,2009-10-07,received,for studies of the structure and function of t...,...,NaN,NaN,NaN,Individual,NaN,NaN,"MRC Laboratory of Molecular Biology, Cambridge...",NaN,NaN,NaN


In [110]:
pd.to_datetime(df.birth_date)

ParserError: month must be in 1..12: 1943-00-00

In [111]:
df['birth_date'] = df.birth_date.str.replace('00','01')

In [112]:
df['bd'] = pd.to_datetime(df.birth_date)

In [116]:
df['bd'].dt.year

0      1943.0
1      1922.0
2      1947.0
3      1926.0
4      1926.0
        ...  
945    1908.0
946    1901.0
947    1867.0
948       NaN
949       NaN
Name: bd, Length: 950, dtype: float64

In [117]:
df['age_awarded'] = df.awardYear - df.bd.dt.year

In [118]:
df[df.birth_countryNow == 'Austria'].\
    sort_values('age_awarded', ascending = False)\
[['category','awardYear','age_awarded','name']]

,category,awardYear,age_awarded,name
503,Physiology or Medicine,1973,87.0,Karl von Frisch
572,Chemistry,2013,83.0,Martin Karplus
683,Literature,2019,77.0,Peter Handke
892,Chemistry,1998,75.0,Walter Kohn
269,Economic Sciences,1974,75.0,Friedrich von Hayek
218,Physiology or Medicine,2000,71.0,Eric Kandel
497,Physiology or Medicine,1927,70.0,Julius Wagner-Jauregg
520,Physiology or Medicine,1973,70.0,Konrad Lorenz
502,Physiology or Medicine,1930,62.0,Karl Landsteiner
726,Chemistry,1925,60.0,Richard Zsigmondy


In [119]:
df[df.birth_countryNow == 'Austria'].sort_values('age_awarded', ascending = False)\
.iloc[0]['name']

'Karl von Frisch'

(5)

In [121]:
df[df.birth_countryNow == "Spain"].shape[0]

7

(6)

In [122]:
current_year = datetime.datetime.now().year

In [123]:
spanish = df[df.birth_countryNow == "Spain"]
spanish

,awardYear,category,categoryFullName,sortOrder,portion,prizeAmount,prizeAmountAdjusted,dateAwarded,prizeStatus,motivation,...,org_founded_locationString,ind_or_org,residence_1,residence_2,affiliation_1,affiliation_2,affiliation_3,affiliation_4,bd,age_awarded
111,1989,Literature,The Nobel Prize in Literature,1,1,3000000,5242311,NaN,received,"for a rich and intensive prose, which with res...",...,NaN,Individual,Spain,NaN,NaN,NaN,NaN,NaN,1916-05-11,73.0
411,1922,Literature,The Nobel Prize in Literature,1,1,122483,3145967,NaN,received,for the happy manner in which he has continued...,...,NaN,Individual,Spain,NaN,NaN,NaN,NaN,NaN,1866-08-12,56.0
480,1904,Literature,The Nobel Prize in Literature,2,1/2,140859,7767960,NaN,received,in recognition of the numerous and brilliant c...,...,NaN,Individual,Spain,NaN,NaN,NaN,NaN,NaN,1832-04-19,72.0
492,1956,Literature,The Nobel Prize in Literature,1,1,200123,2699501,NaN,received,"for his lyrical poetry, which in Spanish langu...",...,NaN,Individual,Spain,NaN,NaN,NaN,NaN,NaN,1881-12-24,75.0
782,1906,Physiology or Medicine,The Nobel Prize in Physiology or Medicine,2,1/2,138536,7421571,NaN,received,in recognition of their work on the structure ...,...,NaN,Individual,NaN,NaN,"Madrid University, Madrid, Spain",NaN,NaN,NaN,1852-05-01,54.0
791,1959,Physiology or Medicine,The Nobel Prize in Physiology or Medicine,1,1/2,220678,2704387,NaN,received,for their discovery of the mechanisms in the b...,...,NaN,Individual,NaN,NaN,"New York University, College of Medicine, New ...",NaN,NaN,NaN,1905-09-24,54.0
883,1977,Literature,The Nobel Prize in Literature,1,1,700000,3080986,NaN,received,for a creative poetic writing which illuminate...,...,NaN,Individual,Spain,NaN,NaN,NaN,NaN,NaN,1898-04-26,79.0


In [124]:
spanish['years_since'] = current_year - spanish.awardYear

C:\Users\priego\AppData\Local\Temp\ipykernel_69244\3096524763.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spanish['years_since'] = current_year - spanish.awardYear


In [125]:
spanish.years_since.min()

33

In [41]:
spanish.sort_values('years_since')[['awardYear','category','name']]

,awardYear,category,name
111,1989,Literature,Camilo José Cela
883,1977,Literature,Vicente Aleixandre
791,1959,Physiology or Medicine,Severo Ochoa
492,1956,Literature,Juan Ramón Jiménez
411,1922,Literature,Jacinto Benavente
782,1906,Physiology or Medicine,Santiago Ramón y Cajal
480,1904,Literature,José Echegaray


(7)

In [126]:
df['decade'] = (df.awardYear//10)*10

In [127]:
df[df.gender == 'female'].groupby('decade').size()

decade
1900     3
1910     1
1920     2
1930     3
1940     3
1960     3
1970     4
1980     4
1990     7
2000    11
2010    13
dtype: int64

In [132]:
df.groupby(['decade','gender']).size().reset_index(name = 'count').\
    query('gender == "female"').drop(columns = 'gender').\
    rename(columns = {"count":"num_women"})

,decade,num_women
0,1900,3
2,1910,1
4,1920,2
6,1930,3
8,1940,3
11,1960,3
13,1970,4
15,1980,4
17,1990,7
19,2000,11


In [43]:
df.groupby(['decade','gender']).size().reset_index(name = 'count').\
query('gender == "female"').reset_index(drop=True).drop(columns = 'gender').\
rename(columns = {"count":"num_women"})

,decade,num_women
0,1900,3
1,1910,1
2,1920,2
3,1930,3
4,1940,3
5,1960,3
6,1970,4
7,1980,4
8,1990,7
9,2000,11


(8)

(a)

In [138]:
df.groupby(['awardYear','category']).size().\
reset_index(name = "num_awardees").\
    groupby('category').num_awardees.mean().\
    reset_index().\
    sort_values('num_awardees')

,category,num_awardees
2,Literature,1.035714
3,Peace,1.340000
1,Economic Sciences,1.647059
0,Chemistry,1.657658
4,Physics,1.884956
5,Physiology or Medicine,1.990909


In [45]:
df.groupby(['awardYear','category']).size().\
reset_index(name = "num_awardees").\
groupby("category").num_awardees.mean()

category
Chemistry                 1.657658
Economic Sciences         1.647059
Literature                1.035714
Peace                     1.340000
Physics                   1.884956
Physiology or Medicine    1.990909
Name: num_awardees, dtype: float64

In [47]:
df.groupby(['awardYear','category']).size().\
groupby('category').mean().nlargest(1).index.values[0]

'Physiology or Medicine'

In [48]:
df.groupby(['awardYear','category']).size().reset_index(name = 'count').\
groupby('category')['count'].mean().reset_index(name = 'mean_n_awardees').\
sort_values('mean_n_awardees', ascending = False).reset_index(drop = True).iloc[0]['category']

'Physiology or Medicine'

(b)

In [49]:
df.groupby(['awardYear','category']).size().reset_index(name = 'count')

,awardYear,category,count
0,1901,Chemistry,1
1,1901,Literature,1
2,1901,Peace,2
3,1901,Physics,1
4,1901,Physiology or Medicine,1
...,...,...,...
592,2019,Economic Sciences,3
593,2019,Literature,1
594,2019,Peace,1
595,2019,Physics,3


In [140]:
shared_prizes = df.groupby(['awardYear','category']).size().\
reset_index(name = 'count').query('count > 1')
shared_prizes.head(10)

,awardYear,category,count
2,1901,Peace,2
7,1902,Peace,2
8,1902,Physics,2
13,1903,Physics,3
16,1904,Literature,2
29,1906,Physiology or Medicine,2
32,1907,Peace,2
37,1908,Peace,2
39,1908,Physiology or Medicine,2
42,1909,Peace,2


In [141]:
shared_prizes.groupby('category').size()

category
Chemistry                 48
Economic Sciences         26
Literature                 4
Peace                     32
Physics                   66
Physiology or Medicine    71
dtype: int64

In [142]:
print(shared_prizes.groupby('category').size().nlargest(1).index.values)

['Physiology or Medicine']


In [58]:
shared_prizes.groupby('category').size()

category
Chemistry                 48
Economic Sciences         26
Literature                 4
Peace                     32
Physics                   66
Physiology or Medicine    71
dtype: int64

(c)

In [143]:
plc = df[df.category == "Literature"].\
groupby("awardYear").filter(lambda x: len(x) > 1)
plc.name
plc.awardYear.unique()

array([1974, 1904, 1917, 1966], dtype=int64)

In [60]:
years_lit_shared = shared_prizes.query('category == "Literature"').awardYear

In [61]:
years_lit_shared

16     1904
72     1917
280    1966
323    1974
Name: awardYear, dtype: int64

In [62]:
df['awardYear'].isin(years_lit_shared)

0      False
1      False
2      False
3      False
4      False
       ...  
945    False
946    False
947    False
948    False
949    False
Name: awardYear, Length: 950, dtype: bool

In [63]:
df[ (df['awardYear'].isin(years_lit_shared)) & (df['category'] == 'Literature')] \
[['awardYear','name','portion','motivation']].sort_values('awardYear')

,awardYear,name,portion,motivation
259,1904,Frédéric Mistral,1/2,in recognition of the fresh originality and tr...
480,1904,José Echegaray,1/2,in recognition of the numerous and brilliant c...
359,1917,Henrik Pontoppidan,1/2,for his authentic descriptions of present-day ...
501,1917,Karl Gjellerup,1/2,"for his varied and rich poetry, which is inspi..."
618,1966,Nelly Sachs,1/2,for her outstanding lyrical and dramatic writi...
796,1966,Shmuel Agnon,1/2,for his profoundly characteristic narrative ar...
237,1974,Eyvind Johnson,1/2,"for a narrative art, far-seeing in lands and a..."
347,1974,Harry Martinson,1/2,for writings that catch the dewdrop and reflec...


(d)

In [145]:
df.groupby(["category","awardYear"]).size().\
    groupby('category').max()

category
Chemistry                 3
Economic Sciences         3
Literature                2
Peace                     3
Physics                   3
Physiology or Medicine    3
dtype: int64

In [64]:
df.groupby(["category","awardYear"]).size().\
groupby('category').max()

category
Chemistry                 3
Economic Sciences         3
Literature                2
Peace                     3
Physics                   3
Physiology or Medicine    3
dtype: int64

In [65]:
df.portion.unique()

array(['1/3', '1', '1/4', '1/2'], dtype=object)

In [66]:
df.groupby(['awardYear','category']).size().reset_index(name = 'count').\
groupby('category')['count'].max().reset_index(name = 'max_n_awardees').\
sort_values('max_n_awardees', ascending = False).reset_index(drop = True)

,category,max_n_awardees
0,Chemistry,3
1,Economic Sciences,3
2,Peace,3
3,Physics,3
4,Physiology or Medicine,3
5,Literature,2


In [67]:
df[df.portion == '1/4'].sort_values('awardYear').head()

,awardYear,category,categoryFullName,sortOrder,portion,prizeAmount,prizeAmountAdjusted,dateAwarded,prizeStatus,motivation,...,ind_or_org,residence_1,residence_2,affiliation_1,affiliation_2,affiliation_3,affiliation_4,bd,age_awarded,decade
566,1903,Physics,The Nobel Prize in Physics,3,1/4,141358,7795478,NaN,received,in recognition of the extraordinary services t...,...,Individual,NaN,NaN,NaN,NaN,NaN,NaN,1867-11-07,36.0,1900
693,1903,Physics,The Nobel Prize in Physics,2,1/4,141358,7795478,NaN,received,in recognition of the extraordinary services t...,...,Individual,NaN,NaN,École municipale de physique et de chimie indu...,NaN,NaN,NaN,1859-05-15,44.0,1900
469,1946,Chemistry,The Nobel Prize in Chemistry,2,1/4,121524,2503929,NaN,received,for their preparation of enzymes and virus pro...,...,Individual,NaN,NaN,"Rockefeller Institute for Medical Research, Pr...",NaN,NaN,NaN,1891-07-05,55.0,1940
897,1946,Chemistry,The Nobel Prize in Chemistry,3,1/4,121524,2503929,NaN,received,for their preparation of enzymes and virus pro...,...,Individual,NaN,NaN,"Rockefeller Institute for Medical Research, Pr...",NaN,NaN,NaN,1904-08-16,42.0,1940
113,1947,Physiology or Medicine,The Nobel Prize in Physiology or Medicine,1,1/4,146115,2914528,NaN,received,for their discovery of the course of the catal...,...,Individual,NaN,NaN,"Washington University, St. Louis, MO, USA",NaN,NaN,NaN,1896-12-05,51.0,1940


In [68]:
df.query('awardYear == 1903 & category == "Physics"')

,awardYear,category,categoryFullName,sortOrder,portion,prizeAmount,prizeAmountAdjusted,dateAwarded,prizeStatus,motivation,...,ind_or_org,residence_1,residence_2,affiliation_1,affiliation_2,affiliation_3,affiliation_4,bd,age_awarded,decade
354,1903,Physics,The Nobel Prize in Physics,1,1/2,141358,7795478,NaN,received,in recognition of the extraordinary services h...,...,Individual,NaN,NaN,"École Polytechnique, Paris, France",NaN,NaN,NaN,1852-12-15,51.0,1900
566,1903,Physics,The Nobel Prize in Physics,3,1/4,141358,7795478,NaN,received,in recognition of the extraordinary services t...,...,Individual,NaN,NaN,NaN,NaN,NaN,NaN,1867-11-07,36.0,1900
693,1903,Physics,The Nobel Prize in Physics,2,1/4,141358,7795478,NaN,received,in recognition of the extraordinary services t...,...,Individual,NaN,NaN,École municipale de physique et de chimie indu...,NaN,NaN,NaN,1859-05-15,44.0,1900


In [69]:
mot = df.query('awardYear == 1903 & category == "Physics"').reset_index()['motivation']
print(mot.iloc[1] == mot.iloc[2])
print(mot.iloc[0] == mot.iloc[1])

True
False
